In [1]:
import numpy as np 
import pandas as pd 
import scipy.stats as stats
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set(style = "whitegrid")

In [26]:
class NaiveBayesClassifier():
     
    def calc_prior(self, features, target):
        self.prior = (features.groupby(target).apply(lambda x:len(x))/self.rows).to_numpy()
        return self.prior
    
    def calc_statistics(self, features, target):
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
        return self.mean, self.var 
    
    
    def gaussian_density(self, class_idx , x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)* ((x-mean)**2)/ (2 * var))
        denominator = np.sqrt(2 * np.pi * var )
        prob = numerator / denominator 
        return prob 
    
    def calc_posterior(self, x):
        posteriors = []
    
        for i in range(self.count):
            prior = np.log(self.prior[i])
            conditional = np.sum(np.log(self.gaussian_density(i, x )))
            posterior = prior + conditional 
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]
  
    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)
        
    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

    def accuracy(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)
        return accuracy

    def visualize(self, y_true, y_pred, target):
        
        tr = pd.DataFrame(data=y_true, columns=[target])
        pr = pd.DataFrame(data=y_pred, columns=[target])
        
        fig, ax = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(15,6))
        
        sns.countplot(x=target, data=tr, ax=ax[0], palette='viridis', alpha=0.7, hue=target, dodge=False)
        sns.countplot(x=target, data=pr, ax=ax[1], palette='viridis', alpha=0.7, hue=target, dodge=False)
        
        fig.suptitle('True vs Predicted Comparison', fontsize=20)

        ax[0].tick_params(labelsize=12)
        ax[1].tick_params(labelsize=12)
        ax[0].set_title("True values", fontsize=18)
        ax[1].set_title("Predicted values", fontsize=18)
        plt.show()
        
        

In [3]:
df = pd.read_csv("C:/Users/Hp/Desktop/ml/irish_flower_classfi/Irish_flower_classfication/Iris.csv")

In [4]:
df = df.sample(frac=1, random_state=1).reset_index(drop=True)  

In [5]:
print(df.shape)

(150, 5)


In [6]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [7]:
X_train, X_test, y_train, y_test = X[:100], X[100:], y[:100],y[100:]

In [8]:
print(X_train.shape, y_train.shape)
print(X_train.shape, y_train.shape)

(100, 4) (100,)
(100, 4) (100,)


In [9]:
X_train.shape

(100, 4)

In [27]:
x = NaiveBayesClassifier()
x.fit(X_train, y_train)

In [41]:
predictions = x.predict(X_test)
x.accuracy(y_test, predictions)
y_test.value_counts(normalize=True)


Iris-setosa        0.38
Iris-versicolor    0.36
Iris-virginica     0.26
Name: species, dtype: float64

3